# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'wisconsin'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
19/19 - 0s - loss: 1.3864 - first_order_loss: 0.6932 - second_order_loss: 0.6931
Epoch 2/50
19/19 - 0s - loss: 1.3859 - first_order_loss: 0.6932 - second_order_loss: 0.6927
Epoch 3/50
19/19 - 0s - loss: 1.3852 - first_order_loss: 0.6930 - second_order_loss: 0.6922
Epoch 4/50
19/19 - 0s - loss: 1.3845 - first_order_loss: 0.6930 - second_order_loss: 0.6914
Epoch 5/50
19/19 - 0s - loss: 1.3829 - first_order_loss: 0.6930 - second_order_loss: 0.6899
Epoch 6/50
19/19 - 0s - loss: 1.3798 - first_order_loss: 0.6929 - second_order_loss: 0.6870
Epoch 7/50
19/19 - 0s - loss: 1.3760 - first_order_loss: 0.6927 - second_order_loss: 0.6833
Epoch 8/50
19/19 - 0s - loss: 1.3703 - first_order_loss: 0.6926 - second_order_loss:

Epoch 27/50
19/19 - 0s - loss: 1.1106 - first_order_loss: 0.6899 - second_order_loss: 0.4214
Epoch 28/50
19/19 - 0s - loss: 1.0910 - first_order_loss: 0.6888 - second_order_loss: 0.4025
Epoch 29/50
19/19 - 0s - loss: 1.0811 - first_order_loss: 0.6885 - second_order_loss: 0.3930
Epoch 30/50
19/19 - 0s - loss: 1.0670 - first_order_loss: 0.6883 - second_order_loss: 0.3786
Epoch 31/50
19/19 - 0s - loss: 1.0906 - first_order_loss: 0.6867 - second_order_loss: 0.4036
Epoch 32/50
19/19 - 0s - loss: 1.0451 - first_order_loss: 0.6877 - second_order_loss: 0.3578
Epoch 33/50
19/19 - 0s - loss: 1.0459 - first_order_loss: 0.6876 - second_order_loss: 0.3581
Epoch 34/50
19/19 - 0s - loss: 1.0264 - first_order_loss: 0.6858 - second_order_loss: 0.3409
Epoch 35/50
19/19 - 0s - loss: 1.0311 - first_order_loss: 0.6862 - second_order_loss: 0.3443
Epoch 36/50
19/19 - 0s - loss: 1.0165 - first_order_loss: 0.6838 - second_order_loss: 0.3326
Epoch 37/50
19/19 - 0s - loss: 1.0576 - first_order_loss: 0.6812 - sec

Epoch 14/50
19/19 - 0s - loss: 1.3055 - first_order_loss: 0.6921 - second_order_loss: 0.6133
Epoch 15/50
19/19 - 0s - loss: 1.2881 - first_order_loss: 0.6920 - second_order_loss: 0.5963
Epoch 16/50
19/19 - 0s - loss: 1.2707 - first_order_loss: 0.6917 - second_order_loss: 0.5790
Epoch 17/50
19/19 - 0s - loss: 1.2550 - first_order_loss: 0.6917 - second_order_loss: 0.5634
Epoch 18/50
19/19 - 0s - loss: 1.2349 - first_order_loss: 0.6912 - second_order_loss: 0.5438
Epoch 19/50
19/19 - 0s - loss: 1.2359 - first_order_loss: 0.6906 - second_order_loss: 0.5450
Epoch 20/50
19/19 - 0s - loss: 1.2033 - first_order_loss: 0.6909 - second_order_loss: 0.5128
Epoch 21/50
19/19 - 0s - loss: 1.1815 - first_order_loss: 0.6899 - second_order_loss: 0.4911
Epoch 22/50
19/19 - 0s - loss: 1.1662 - first_order_loss: 0.6900 - second_order_loss: 0.4763
Epoch 23/50
19/19 - 0s - loss: 1.1599 - first_order_loss: 0.6893 - second_order_loss: 0.4708
Epoch 24/50
19/19 - 0s - loss: 1.1435 - first_order_loss: 0.6892 - sec

19/19 - 0s - loss: 1.3863 - first_order_loss: 0.6932 - second_order_loss: 0.6931
Epoch 2/50
19/19 - 0s - loss: 1.3859 - first_order_loss: 0.6931 - second_order_loss: 0.6928
Epoch 3/50
19/19 - 0s - loss: 1.3854 - first_order_loss: 0.6930 - second_order_loss: 0.6924
Epoch 4/50
19/19 - 0s - loss: 1.3846 - first_order_loss: 0.6930 - second_order_loss: 0.6916
Epoch 5/50
19/19 - 0s - loss: 1.3828 - first_order_loss: 0.6929 - second_order_loss: 0.6898
Epoch 6/50
19/19 - 0s - loss: 1.3802 - first_order_loss: 0.6929 - second_order_loss: 0.6873
Epoch 7/50
19/19 - 0s - loss: 1.3767 - first_order_loss: 0.6928 - second_order_loss: 0.6839
Epoch 8/50
19/19 - 0s - loss: 1.3714 - first_order_loss: 0.6927 - second_order_loss: 0.6787
Epoch 9/50
19/19 - 0s - loss: 1.3635 - first_order_loss: 0.6927 - second_order_loss: 0.6707
Epoch 10/50
19/19 - 0s - loss: 1.3507 - first_order_loss: 0.6926 - second_order_loss: 0.6581
Epoch 11/50
19/19 - 0s - loss: 1.3378 - first_order_loss: 0.6925 - second_order_loss: 0.64

Epoch 39/50
19/19 - 0s - loss: 1.0023 - first_order_loss: 0.6848 - second_order_loss: 0.3177
Epoch 40/50
19/19 - 0s - loss: 1.0129 - first_order_loss: 0.6830 - second_order_loss: 0.3290
Epoch 41/50
19/19 - 0s - loss: 0.9982 - first_order_loss: 0.6843 - second_order_loss: 0.3138
Epoch 42/50
19/19 - 0s - loss: 0.9927 - first_order_loss: 0.6837 - second_order_loss: 0.3090
Epoch 43/50
19/19 - 0s - loss: 1.0374 - first_order_loss: 0.6781 - second_order_loss: 0.3585
Epoch 44/50
19/19 - 0s - loss: 0.9954 - first_order_loss: 0.6817 - second_order_loss: 0.3151
Epoch 45/50
19/19 - 0s - loss: 0.9830 - first_order_loss: 0.6799 - second_order_loss: 0.3019
Epoch 46/50
19/19 - 0s - loss: 1.0158 - first_order_loss: 0.6847 - second_order_loss: 0.3327
Epoch 47/50
19/19 - 0s - loss: 0.9809 - first_order_loss: 0.6811 - second_order_loss: 0.2999
Epoch 48/50
19/19 - 0s - loss: 0.9851 - first_order_loss: 0.6808 - second_order_loss: 0.3045
Epoch 49/50
19/19 - 0s - loss: 1.0032 - first_order_loss: 0.6721 - sec

Epoch 27/50
19/19 - 0s - loss: 1.0864 - first_order_loss: 0.6891 - second_order_loss: 0.3972
Epoch 28/50
19/19 - 0s - loss: 1.0693 - first_order_loss: 0.6884 - second_order_loss: 0.3811
Epoch 29/50
19/19 - 0s - loss: 1.0634 - first_order_loss: 0.6889 - second_order_loss: 0.3746
Epoch 30/50
19/19 - 0s - loss: 1.0594 - first_order_loss: 0.6875 - second_order_loss: 0.3717
Epoch 31/50
19/19 - 0s - loss: 1.0940 - first_order_loss: 0.6859 - second_order_loss: 0.4082
Epoch 32/50
19/19 - 0s - loss: 1.0479 - first_order_loss: 0.6873 - second_order_loss: 0.3610
Epoch 33/50
19/19 - 0s - loss: 1.0333 - first_order_loss: 0.6865 - second_order_loss: 0.3467
Epoch 34/50
19/19 - 0s - loss: 1.0328 - first_order_loss: 0.6854 - second_order_loss: 0.3480
Epoch 35/50
19/19 - 0s - loss: 1.0254 - first_order_loss: 0.6855 - second_order_loss: 0.3395
Epoch 36/50
19/19 - 0s - loss: 1.0190 - first_order_loss: 0.6847 - second_order_loss: 0.3345
Epoch 37/50
19/19 - 0s - loss: 1.0393 - first_order_loss: 0.6804 - sec